In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()*_input.size(0) / (counter * train_loader.batch_size)))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 4

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(4):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
23012,"[1830.4840087890625, 441.3025817871094]","[1829.257080078125, 440.17523193359375]","[1828.089599609375, 439.2045593261719]","[1826.897705078125, 438.1288757324219]","[1825.719482421875, 437.0605773925781]","[1824.5487060546875, 435.9916076660156]","[1823.3695068359375, 434.9254150390625]","[1822.18115234375, 433.8665466308594]","[1820.9854736328125, 432.8179016113281]","[1819.783935546875, 431.7805480957031]",...,"[1806.407470703125, 420.56982421875]","[1805.193115234375, 419.54132080078125]","[1803.9881591796875, 418.52685546875]","[1802.800537109375, 417.53369140625]","[1801.6390380859375, 416.56781005859375]","[1800.5115966796875, 415.6333312988281]","[1799.423828125, 414.73199462890625]","[1798.378662109375, 413.8638000488281]","[1797.3763427734375, 413.02728271484375]","[1796.415771484375, 412.2203063964844]"
14709,"[731.9102172851562, 1146.4796142578125]","[732.2720947265625, 1146.274658203125]","[732.7483520507812, 1146.3294677734375]","[733.2298583984375, 1146.3115234375]","[733.7185668945312, 1146.2803955078125]","[734.2092895507812, 1146.24560546875]","[734.7017211914062, 1146.2086181640625]","[735.1943359375, 1146.171142578125]","[735.6857299804688, 1146.1343994140625]","[736.1742553710938, 1146.099365234375]",...,"[741.29345703125, 1146.0135498046875]","[741.76953125, 1146.05078125]","[742.2545166015625, 1146.0955810546875]","[742.748046875, 1146.145751953125]","[743.2484741210938, 1146.197998046875]","[743.7534790039062, 1146.24853515625]","[744.2601318359375, 1146.2933349609375]","[744.765380859375, 1146.328369140625]","[745.266357421875, 1146.349853515625]","[745.7606811523438, 1146.354736328125]"
5321,"[2061.024169921875, 625.3036499023438]","[2061.71484375, 626.1182861328125]","[2062.645751953125, 626.9326171875]","[2063.7041015625, 627.6058349609375]","[2064.7373046875, 628.2954711914062]","[2065.784912109375, 629.032470703125]","[2066.853271484375, 629.7959594726562]","[2067.92626953125, 630.5757446289062]","[2068.994140625, 631.3645629882812]","[2070.051025390625, 632.1541137695312]",...,"[2082.634765625, 641.470947265625]","[2083.8662109375, 642.3850708007812]","[2084.958251953125, 643.2482299804688]","[2085.915283203125, 644.0587768554688]","[2086.74951171875, 644.815673828125]","[2087.474853515625, 645.519775390625]","[2088.10595703125, 646.1734619140625]","[2088.655029296875, 646.7798461914062]","[2089.133544921875, 647.3423461914062]","[2089.550537109375, 647.8643188476562]"
31292,"[418.3989562988281, 1096.62890625]","[418.20904541015625, 1096.9771728515625]","[418.262939453125, 1097.6185302734375]","[418.2546081542969, 1098.1815185546875]","[418.2633361816406, 1098.7777099609375]","[418.2565612792969, 1099.3453369140625]","[418.2540283203125, 1099.8994140625]","[418.2513732910156, 1100.4329833984375]","[418.2528991699219, 1100.9505615234375]","[418.25909423828125, 1101.453125]",...,"[418.7197570800781, 1106.5079345703125]","[418.78607177734375, 1106.9552001953125]","[418.8539123535156, 1107.40283203125]","[418.92266845703125, 1107.8502197265625]","[418.9916076660156, 1108.2967529296875]","[419.059814453125, 1108.741455078125]","[419.1263122558594, 1109.1829833984375]","[419.1899719238281, 1109.619873046875]","[419.2496643066406, 1110.0506591796875]","[419.3042907714844, 1110.4735107421875]"
11653,"[2140.729736328125, 688.213134765625]","[2142.2578125, 689.5001831054688]","[2143.3330078125, 690.3375244140625]","[2144.525390625, 691.224853515625]","[2145.780029296875, 692.1748046875]","[2147.064208984375, 693.1905517578125]","[2148.37353515625, 694.2600708007812]","[2149.701904296875, 695.36767578125]","[2151.0419921875, 696.495361328125]","[2152.3837890625, 697.623779296875]",...,"[2166.533203125, 709.5028076171875]","[2168.185546875, 710.9403686523438]","[2169.626220703125, 712.251220703125]","[2170.808837890625, 713.4075317382812]","[2171.761474609375, 714.4169311523438]","[2172.529052734375, 715.2977294921875]","[2173.15185546875, 716.068

In [ ]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
23012,1830.484009,441.302582,1829.257080,440.175232,1828.089600,439.204559,1826.897705,438.128876,1825.719482,437.060577,...,1800.511597,415.633331,1799.423828,414.731995,1798.378662,413.863800,1797.376343,413.027283,1796.415771,412.220306
14709,731.910217,1146.479614,732.272095,1146.274658,732.748352,1146.329468,733.229858,1146.311523,733.718567,1146.280396,...,743.753479,1146.248535,744.260132,1146.293335,744.765381,1146.328369,745.266357,1146.349854,745.760681,1146.354736
5321,2061.024170,625.303650,2061.714844,626.118286,2062.645752,626.932617,2063.704102,627.605835,2064.737305,628.295471,...,2087.474854,645.519775,2088.105957,646.173462,2088.655029,646.779846,2089.133545,647.342346,2089.550537,647.864319
31292,418.398956,1096.628906,418.209045,1096.977173,418.262939,1097.618530,418.254608,1098.181519,418.263336,1098.777710,...,419.059814,1108.741455,419.126312,1109.182983,419.189972,1109.619873,419.249664,1110.050659,419.304291,1110.473511
11653,2140.729736,688.213135,2142.257812,689.500183,2143.333008,690.337524,2144.525391,691.224854,2145.780029,692.174805,...,2172.529053,715.297729,2173.151855,716.068665,2173.661377,716.746338,2174.081299,717.344421,2174.429688,717.874390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35984,303.540771,1139.383179,302.783081,1139.459351,301.922272,1139.543579,301.123291,1139.643433,300.318756,1139.719360,...,282.723511,1140.127319,281.890320,1140.145874,281.049042,1140.166138,280.197998,1140.188599,279.335663,1140.213623
7166,699.215088,722.477905,699.323792,722.458740,699.519348,722.447388,699.724243,722.429932,699.946960,722.396179,...,707.544495,721.939758,707.883728,721.907410,708.209778,721.862000,708.522888,721.802612,708.823364,721.728455
32003,579.476868,1386.086670,578.643799,1386.004028,577.847107,1386.052979,577.087341,1386.114990,576.308044,1386.151367,...,558.912537,1385.311157,558.055359,1385.251099,557.186279,1385.187622,556.303467,1385.120239,555.405457,1385.048340


In [12]:
df2.to_csv("outputs.csv", index=True)